# Baseline implementation of relevant error rate-based change-detection algorithms

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Loading

In [3]:
# Load data
train1 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/train1.csv")
train2 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/train2.csv")
test1 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/test1.csv")
test2 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/test2.csv")
label1 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/label1.csv")
label2 = pd.read_csv("/Users/emmatosato/Documents/UNI Locale/Erasmus/OST/ost-sm-change-detection/data_analysis/preprocessed_data/label2.csv")

## Functions

### Evaluation

In [ ]:
# Definitions of functions
def calculate_delay_of_detection(true_change_indexes, detected_indexes):
    delays = [index - detected_index for detected_index, index in zip(detected_indexes, true_change_indexes)]
    average_delay = sum(delays) / len(delays)
    return average_delay

def calculate_false_detection_rate(true_change_indexes, detected_indexes):
    total_drifts = len(true_change_indexes)
    total_detected = len(detected_indexes)
    false_detections = total_detected - total_drifts
    fdr = false_detections / total_drifts
    return fdr

def calculate_miss_detection_rate(true_change_indexes, detected_indexes):
    total_drifts = len(true_change_indexes)
    total_detected = len(detected_indexes)
    mdr = (total_drifts - total_detected) / total_drifts
    return mdr

def calculate_rate_of_drift(detected_indexes, total_time):
    total_detected = len(detected_indexes)
    rod = total_detected / total_time
    return rod

## Classification model

In [23]:
# Split the data into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(test1.iloc[:,1:], label1['label'], test_size=0.2, random_state=42)
stream_size = test_data.shape[0]
true_attack_indexes = indices_of_ones = test_labels[test_labels['labels'] == 1].index.values

# Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_data, train_labels)

# Make predictions on the test set
rf_model_predictions = rf_model.predict(test_data)

In [ ]:
true_attack_indexes = indices_of_ones = test_labels[test_labels['labels'] == 1].index.values

## CUMSUM Algorithm

The cumulative sum (CUSUM) test is designed to give an alarm when the mean of the input data significantly deviates from its previous value.

*References :*
- Machine Learning for Data Streams book
- Thomas Flynn and Shinjae Yoo. *Change Detection with the Kernel Cumulative Sum Algorithm*

In [22]:
class CUSUM:
    def __init__(self, k=0.5, h=5):
        self.k = k
        self.h = h
        self.g = 0

    def add_element(self, error):
        zt = error
        self.g = max(0, self.g + zt - self.k)

    def check_drift(self):
        return self.g > self.h


In [ ]:
# Initialize CUSUM for change detection
cusum = CUSUM()  

detected_indexes_cusum = []

for i in range(stream_size):
    prediction = 0 if i < stream_size // 2 else 1
    error = int(prediction != test_labels[i])

   # Update CUSUM with the error signal
    cusum.add_element(error)

    # Check for change detection
    if cusum.detected_change():
        detected_indexes_cusum.append(i)


In [ ]:
# Evaluate change detection performance using CUSUM
average_delay_cusum = calculate_delay_of_detection(true_change_indexes, detected_indexes_cusum)
fdr_cusum = calculate_false_detection_rate(true_change_indexes, detected_indexes_cusum)
mdr_cusum = calculate_miss_detection_rate(true_change_indexes, detected_indexes_cusum)
rod_cusum = calculate_rate_of_drift(detected_indexes_cusum, total_time=stream_size - 500)


## ADaptive WINdowing (ADWIN)

**ADWIN (ADaptive WINdowing)** is an adaptive sliding window algorithm for detecting change, and keeping updated statistics about a data stream.
ADWIN allows algorithms not adapted for drifting data, to be resistant to this phenomenon.

The general idea is to keep statistics from a window of variable size while detecting concept drift.

The algorithm will decide the size of the window by cutting the statistics' window at different points and analysing the average of some statistic over these two windows. If the absolute value of the difference between the two averages surpasses a pre-defined threshold, change is detected at that point and all data before that time is discarded.

*References*: 
Thomas, Flynn, and Yoo Shinjae. Change Detection with the Kernel Cumulative Sum Algorithm. ArXiv E-prints, 2017, https://doi.org/10.48550/arXiv.1903.01661.

In [33]:
class ADWIN:
    def __init__(self, delta=0.002):
        self.delta = delta
        self.window = []
        self.total_count = 0

    def add_element(self, error):
        self.window.append(error)
        self.total_count += 1

        if len(self.window) > 1:
            self.check_drift()

    def check_drift(self):
        num_buckets = len(self.window)

        for i in range(1, num_buckets):
            w0 = self.window[:i]
            w1 = self.window[i:]

            if self.test(w0, w1):
                self.window.pop(0)
                break

    def test(self, w0, w1):
        mean_w0 = sum(w0) / len(w0) if len(w0) > 0 else 0
        mean_w1 = sum(w1) / len(w1) if len(w1) > 0 else 0

        var_w0 = sum((x - mean_w0) ** 2 for x in w0) / len(w0) if len(w0) > 1 else 0
        var_w1 = sum((x - mean_w1) ** 2 for x in w1) / len(w1) if len(w1) > 1 else 0

        if mean_w1 == 0:
            return False

        p_value = (mean_w0 + 2 * (var_w0 ** 0.5)) / (mean_w1 + 2 * (var_w1 ** 0.5))
        
        return p_value < self.delta



In [34]:
# Initialize ADWIN for change detection
adwin = ADWIN(delta=0.002)

# List detected change points
detected_indexes_adwin = []

for i in range(stream_size):
    error = int(rf_model_predictions[i] != test_labels.iloc[i])

    # Update ADWIN with the error signal
    adwin.add_element(error)

    # Check for change detection
    if adwin.check_drift():
        detected_indexes_adwin.append(i)

KeyboardInterrupt: 

In [ ]:
# Evaluate change detection performance using ADWIN
average_delay_adwin = calculate_delay_of_detection(true_attack_indexes, detected_indexes_adwin)
fdr_adwin = calculate_false_detection_rate(true_attack_indexes, detected_indexes_adwin)
mdr_adwin = calculate_miss_detection_rate(true_attack_indexes, detected_indexes_adwin)
#rod_adwin = calculate_rate_of_drift(detected_indexes_adwin, total_time=stream_size - 500)

## Early Drift Detection Method (EDDM)

This method works by keeping track of the average distance between two errors instead of only the error rate. For this, it is necessary to keep track of the running average distance and the running standard deviation, as well as the maximum distance and the maximum standard deviation.

The algorithm works similarly to the DDM algorithm. Like DDM, there are two threshold values that define the borderline between no change, warning zone, and drift detected.
These are as follows:

**Warning zone**
$$ \frac{(p_i + 2 * s_i)}{(p_{max} + 2 * s_{max})} < \alpha $$ 

**Change detected**

$$\frac{(p_i + 2 * s_i)}{(p_{max} + 2 * s_{max})} < \beta $$ 




$\alpha$ and  $\beta$ are set to 0.95 and 0.9, respectively.


<br><br>

*References:*
Early Drift Detection Method. Manuel Baena-Garcia, Jose Del Campo-Avila, Raúl Fidalgo, Albert Bifet, Ricard Gavalda, Rafael Morales-Bueno. In Fourth International Workshop on Knowledge Discovery from Data Streams, 2006.

In [1]:
class EDDM:
    def __init__(self, alpha=0.95, beta=0.9, min_num_instances=30):
        self.alpha = alpha
        self.beta = beta
        self.min_num_instances = min_num_instances

    def add_element(self, prediction):
        self.in_concept_change = 0
        self.n += 1

        if prediction == 1.0:
            self.in_warning_zone = 0
            self.delay = 0
            self.num_errors += 1
            self.last_d = self.d
            self.d = self.n - 1
            distance = self.d - self.last_d
            old_mean = self.mean
            self.mean = self.mean + (float(distance) - self.mean) / self.num_errors
            self.estimation = self.mean
            self.std_temp = self.std_temp + (distance - self.mean) * (distance - old_mean)
            std = np.sqrt(self.std_temp / self.num_errors)
            m2s = self.mean + 2 * std

            if self.n < self.min_num_instances:
                return

            if m2s > self.m2s_max:
                self.m2s_max = m2s
            else:
                p = m2s / self.m2s_max
                if (self.num_errors > self.min_num_instances) and (p < self.alpha):
                    self.in_concept_change = 1

                elif (self.num_errors > self.min_num_instances) and (p < self.beta):
                    self.in_warning_zone = 1

                else:
                    self.in_warning_zone = 0

    def detected_warning_zone(self):
        return self.in_warning_zone

    def detected_change(self):
        return self.in_concept_change


In [ ]:
# Initialize EDDM for change detection
eddm = EDDM()

# List to store detected change points using EDDM
detected_indexes_eddm = []

for i in range(stream_size):
    prediction = 0 if i < stream_size // 2 else 1
    error = int(prediction != stream_labels[i])

    # Update EDDM
    eddm.add_element(error)

    # Check for change detection
    if eddm.detected_change():
        detected_indexes_eddm.append(i)

In [ ]:
# Evaluate change detection performance using EDDM
average_delay_eddm = calculate_delay_of_detection(true_change_indexes, detected_indexes_eddm)
fdr_eddm = calculate_false_detection_rate(true_change_indexes, detected_indexes_eddm)
mdr_eddm = calculate_miss_detection_rate(true_change_indexes, detected_indexes_eddm)
rod_eddm = calculate_rate_of_drift(detected_indexes_eddm, total_time=stream_size - 500)